# Introduction

In this notebook we are going to explore the efficacy of *ensemble models* on a classification problem based on the `make_circles` dataset available in SKLearn.

Inside, we will create examples of some of the concepts we discussed in the lecture and plot the outputs.

Specifically we will explore:
  * Hard voting aggregation of varied algorithms
  * Soft voting aggregation of varied algorithms
  * Stacking
  * Bagging
  * Boosting

Please note that the base machine learning algorithms in this notebook have been throttled down significantly in order to better showcase the improvement that ensemble models can provide. In reality, this dataset is sufficiently simple that even trivial hyperparameter tuning can result in significantly better prediction accuracies than you see here!

I have tried to be as fair as possible (e.g. comparing a rubbish decision tree with a random forest composed of rubbish decision trees), but some of the most egregious things you can expect to see here:
  * Terrible train/test splits
  * Horrible choices of algorithms (LogReg and LinearSVM for circles?!)
  * Extremely shallow trees (Making a single decision point for 2D data necessarily leads to a straight line -- not a great way of identifying circles)
  
Hopefully, despite the handicaps our poor models are subjected to, the main points of why ensemble models are powerful will be apparent.

# Initialisation

Let's import the packages we need.

I also turn off warnings here -- I'm running an old version of SKLearn here (0.20.3) and it's very vocal about default behaviours and futureproofing your code. They're just warnings so if you see them in your own code, you don't have to worry about them, I just found them distracting so turned them off.

The final line is a bit of Jupyter "cell magic" to plot the figures in-line.

In [ ]:
import matplotlib.pyplot
import numpy
import sklearn.calibration
import sklearn.datasets
import sklearn.ensemble
import sklearn.linear_model
import sklearn.metrics
import sklearn.neighbors
import sklearn.svm
import sklearn.tree
import warnings
warnings.filterwarnings("ignore")

%matplotlib inline

The following function sets up the main plotting used in this notebook to visualise the full dataset we are exploring, as well as any decision boundaries calculated by each algorithm.

In [ ]:
def plot_circles(coordinates, circle_id, title=None, decision_region=None):
    matplotlib.pyplot.figure(figsize=(10, 10), facecolor="white")
    
    # Plot main data
    matplotlib.pyplot.scatter(coordinates[circle_id == 0][:, 0], coordinates[circle_id == 0][:, 1], c="r", label="circle 1")
    matplotlib.pyplot.scatter(coordinates[circle_id == 1][:, 0], coordinates[circle_id == 1][:, 1], c="b", label="circle 2")
    
    # Plot decision regions
    if decision_region is not None:
        (xx, yy, Z) = decision_region
        matplotlib.pyplot.contourf(xx, yy, Z, cmap=matplotlib.pyplot.cm.seismic_r, alpha=0.4)
        
    
    matplotlib.pyplot.xlabel("X Coordinate (Arb. U.)", fontsize=28)
    matplotlib.pyplot.ylabel("Y Coordinate (Arb. U.)", fontsize=28)
    matplotlib.pyplot.legend(fontsize=16)
    if title is not None:
        matplotlib.pyplot.title(title, fontsize=18)
    matplotlib.pyplot.show()

# Generate Data

Let's make and visualise our input data!

In this case we'll be generating a set of 1000 datapoints that are classified into an inner circle and an outer annulus with double the radius.

We also add in some Gaussian noise to the coordinates to make it a little less clean, with a standard deviation of 0.3 arbitrary units.

In [ ]:
coordinates, circle_id = sklearn.datasets.make_circles(n_samples=1000, noise=0.3, factor=0.5, random_state=42)
print("Coordinates {} = {}".format(coordinates.shape, coordinates))
print("Circle {} = {}".format(circle_id.shape, circle_id))
plot_circles(coordinates, circle_id)

We will now split the data. As I mentioned before, I'm going to hamstring my models by only providing them 10% of the data to train on. This will make them all rubbish to show you how even the weakest predictors can still provide good results when combined correctly in an ensemble.

In [ ]:
X_train, X_test, y_train, y_test = sklearn.model_selection.train_test_split(coordinates, circle_id, train_size=100, random_state=42)

# Hard and Soft Voting Aggregation

For this section, we will use 3 different models, none of which are particularly good at solving this problem:
  * LR -- A Logistic Regression model
  * DT -- A Decision Tree
  * SVM -- A Linear Support Vector Classifier
  
Rather than just using `sklearn.svm.LinearSVC`, we nestle it within a `sklearn.calibration.CalibratedClassifierCV`. This is because `LinearSVC` doesn't output the prediction probabilities by default. Wrapping it allows us to include it in our soft-voting ensemble, which requires some measure of confidence in the classification in order to judge how much (or little) to weight the SVM's prediction.

In [ ]:
LR = sklearn.linear_model.LogisticRegression(random_state=42)
DT = sklearn.tree.DecisionTreeClassifier(random_state=42, max_depth=5)
SVM = sklearn.calibration.CalibratedClassifierCV(sklearn.svm.LinearSVC(random_state=42))

voting_hard = sklearn.ensemble.VotingClassifier(
    estimators=[("LR", LR), ("DT", DT), ("SVM", SVM)],# ("KN", KN)],
    voting="hard"
)

voting_soft = sklearn.ensemble.VotingClassifier(
    estimators=[("LR", LR), ("DT", DT), ("SVM", SVM)],# ("KN", KN)],
    voting="soft"
)

models = {
    "Logistic Regression": LR,
    "Decision Tree": DT,
    "Linear Support Vector Machine": SVM,
    "Voting (Hard)": voting_hard,
    "Voting (Soft)": voting_soft,
}

accuracy = []
for model_name, model in models.items():
    model.fit(X_train, y_train)
    predictions = model.predict(X_test)
    accuracy.append(sklearn.metrics.accuracy_score(y_test, predictions))
    # Obtain the decision regions for plotting
    x_min, x_max = X_test[:, 0].min() - 1, X_test[:, 0].max() + 1
    y_min, y_max = X_test[:, 1].min() - 1, X_test[:, 1].max() + 1
    xx, yy = numpy.meshgrid(
        numpy.arange(x_min, x_max, 0.02),
        numpy.arange(y_min, y_max, 0.02)
    )
    Z = model.predict(numpy.c_[xx.ravel(), yy.ravel()]).reshape(xx.shape)
    
    plot_circles(
        coordinates,
        circle_id,
        title="Predictions from {} (Accuracy = {:.4f})".format(model_name, accuracy[-1]),
        decision_region=(xx, yy, Z)
    )

print("---=== ACCURACY SUMMARY ===---")
for model_index, model_name in enumerate(models.keys()):
    print(model_name, "{:.4f}".format(accuracy[model_index]))

As you can see, we have pretty varied results from each classifier. The decision tree did pretty well compared to the other two.

Look at the decision surfaces for the hard and soft voting classifiers though. With the hard voting, you can clearly see the decision surface shapes corresponding to the LR at the top, the DT on the right and the SVM on the left. Each quadrant is dominated by a single model. It's picked the model that did best in each quadrant, but has not in any way mixed the decision surfaces to get a more accurate prediction.

The soft classifier on the other hand has produced a pretty interesting decision surface. It's easy to pick out the shape of the DT surface from the right hand side of the result, but on the left there are components of the surface that are not directly present in any of the other surfaces.

It is this ability to interpolate a new decision surface that can boost the soft-classifier's accuracy.

For this example neither of the ensemble models outperformed the decision tree, but there was a signficant improvement from using soft-voting to aggregate the predictions than hard-voting.

# Stacking

Note: The `sklearn.ensemble.StackingClassifier` function can be used here instead of performing the stacking manually as below, but I wanted to show you the process.

In order to employ stacking, we need to first split our training data into two subsets.

In [ ]:
subset_X1, subset_X2, subset_y1, subset_y2 = sklearn.model_selection.train_test_split(X_train, y_train, train_size=80, random_state=42)

Let's set up the models we need:

In [ ]:
LR1 = sklearn.linear_model.LogisticRegression(random_state=42)
DT1 = sklearn.tree.DecisionTreeClassifier(random_state=42, max_depth=5)
SVM1 = sklearn.svm.LinearSVC(random_state=42)

layer_1 = {
    "Logistic Regression": LR1,
    "Decision Tree": DT1,
    "Linear Support Vector Machine": SVM1,
}

blender = sklearn.linear_model.LogisticRegression(random_state=42)

Now, we can fit each model to the first subset, and generate predictions for the second subset. We form a `predictions_array`, and will build the training data for the blender from this.

In [ ]:
predictions_array_train = []
accuracies = []
for model_name, model in layer_1.items():
    model.fit(subset_X1, subset_y1)
    predictions_array_train.append(model.predict(subset_X2))
    accuracies.append(sklearn.metrics.accuracy_score(subset_y2, predictions_array_train[-1]))

blender_input = numpy.array(list(zip(*predictions_array_train)))

Now we can fit the blender (a Logitistic Regression Model) to the input data, knowing it should fit to `subset_y2` defined above

In [ ]:
blender.fit(blender_input, subset_y2)

predictions_array_test = []
for model_name, model in layer_1.items():
    predictions_array_test.append(model.predict(X_test))
blender_test_inputs = numpy.array(list(zip(*predictions_array_test)))
    
predictions = blender.predict(blender_test_inputs)
accuracy = sklearn.metrics.accuracy_score(y_test, predictions)

for index, (feature, coeff) in enumerate(list(zip(list(layer_1.keys()) + ["INTERCEPT"], list(blender.coef_[0]) + list(blender.intercept_)))):
    try:
        print("Coefficient for {} = {:.4f} (Accuracy Metric = {:.4f})".format(feature, coeff, accuracies[index]))
    except IndexError:
        print("Coefficient for {} = {:.4f}".format(feature, coeff))
        
    
print("\nStacked Ensemble Accuracy = {:.4f}".format(accuracy))

By training the Blender on the second layer, the Decision tree is weighted more highly than the other two models. However, all models actively contribute to the final ensemble accuracy, and so the ensemble performs better than any of the consituent models themselves.

# Bagging

Note: A classifier consisting of multiple Decision Tree estimators with bagging of training data is effectively the same as a Random Forest. In the below code, we could use the useful SKLearn helper class `RandomForestClassifier` instead.

Let's take a look at an example of Bagging using an ensemble of 200 decision trees:

In [ ]:
model_tree = sklearn.tree.DecisionTreeClassifier(random_state=42, max_depth=1)
model_forest = sklearn.ensemble.BaggingClassifier(
    sklearn.tree.DecisionTreeClassifier(random_state=424, max_depth=1),
    n_estimators=50,
    bootstrap=True,
    n_jobs=-1,
)

In [ ]:
accuracy = []
for model in (model_tree, model_forest):
    model.fit(X_train, y_train)
    predictions = model.predict(X_test)
    accuracy.append(sklearn.metrics.accuracy_score(y_test, predictions))
    
    # Obtain the decision regions for plotting
    x_min, x_max = X_test[:, 0].min() - 1, X_test[:, 0].max() + 1
    y_min, y_max = X_test[:, 1].min() - 1, X_test[:, 1].max() + 1
    xx, yy = numpy.meshgrid(
        numpy.arange(x_min, x_max, 0.02),
        numpy.arange(y_min, y_max, 0.02)
    )
    Z = model.predict(numpy.c_[xx.ravel(), yy.ravel()]).reshape(xx.shape)
    
    plot_circles(
        coordinates,
        circle_id,
        title="Predictions from {} (Accuracy = {:.4f})".format(model.__class__.__name__, accuracy[-1]),
        decision_region=(xx, yy, Z)
    )

print("The Random Forest is a {:.2f}% improvement over the Decision Tree!".format((100 * (accuracy[1] - accuracy[0]) / accuracy[0])))

Given that each decision tree makes a single decision and so can divide the data plane into two decision surfaces, an ensemble of 50 trees can make multiple cuts, increasing the accuracy of the model.

# Boosting

Instead of just bagging, we can also do some Boosting using the AdaBoost algorithm we discussed in the presentation.

If we use the same terrible trees, but this time instead of restricting their training data (or feature set, although that's kind of a moot point when there are only 2 input features), and use the same number of estimators for our Forest (50), can we obtain an improvement by training them sequentially to address the shortcomings of the previous predictor rather than in parallel like above?

In [ ]:
model_ada = sklearn.ensemble.AdaBoostClassifier(
    sklearn.tree.DecisionTreeClassifier(random_state=4242, max_depth=1),
    n_estimators=50,
    algorithm="SAMME.R",
    learning_rate=0.5,
)

In [ ]:
accuracy = []
for model in (model_tree, model_ada):
    model.fit(X_train, y_train)
    predictions = model.predict(X_test)
    accuracy.append(sklearn.metrics.accuracy_score(y_test, predictions))
    
    # Obtain the decision regions for plotting
    x_min, x_max = X_test[:, 0].min() - 1, X_test[:, 0].max() + 1
    y_min, y_max = X_test[:, 1].min() - 1, X_test[:, 1].max() + 1
    xx, yy = numpy.meshgrid(
        numpy.arange(x_min, x_max, 0.02),
        numpy.arange(y_min, y_max, 0.02)
    )
    Z = model.predict(numpy.c_[xx.ravel(), yy.ravel()]).reshape(xx.shape)
    
    plot_circles(
        coordinates,
        circle_id,
        title="Predictions from {} (Accuracy = {:.4f})".format(model.__class__.__name__, accuracy[-1]),
        decision_region=(xx, yy, Z)
    )

print("The Ada Boosted Forest is a {:.2f}% improvement over the Decision Tree!".format((100 * (accuracy[1] - accuracy[0]) / accuracy[0])))

The performance improvement is huge! Before, most of the 50 trees ended up being the same because they could only make one decision on one of the two input features, and so most of the trees actually looked idential. This time, by weighting the training instances for each predictor, the trees end up being much more diverse, which allows for significantly better predictive capability!